# Task 2: Exploratory Data Analysis

**EthioPulse-Forecaster - Financial Inclusion Forecasting System**

This notebook performs comprehensive exploratory data analysis on Ethiopia's financial inclusion trajectory.

## Objectives
1. Analyze Access and Usage trends (2011-2024)
2. Explain the 2021-2024 slowdown
3. Examine infrastructure and enabler variables
4. Overlay events on trends
5. Perform correlation analysis
6. Extract at least 5 policy-relevant insights
7. Document data gaps and hypotheses for forecasting

In [ ]:
# Import libraries
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

sys.path.append(str(Path.cwd().parent))
from src.data_utils import (
    load_unified_data,
    get_observations_by_pillar,
    get_events_by_year
)

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

DATA_DIR = Path.cwd().parent / "data"
PROCESSED_DATA_PATH = DATA_DIR / "processed" / "ethiopia_fi_enriched.csv"
FIGURES_DIR = Path.cwd().parent / "reports" / "figures"
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

print("✓ Libraries imported")

## 1. Load Enriched Dataset

In [ ]:
# Load enriched data (check both CSV and Excel formats)
PROCESSED_DATA_XLSX = DATA_DIR / "processed" / "ethiopia_fi_enriched.xlsx"
PROCESSED_DATA_XLSX_ALT = DATA_DIR / "processed" / "ethiopia_fi_unified_data.xlsx"

df = pd.DataFrame()
data_loaded = False

# Try to load from Task 1 output (CSV)
if PROCESSED_DATA_PATH.exists():
    try:
        df = pd.read_csv(PROCESSED_DATA_PATH)
        print(f"✓ Loaded {len(df):,} records from {PROCESSED_DATA_PATH}")
        data_loaded = True
    except Exception as e:
        print(f"⚠ Error loading {PROCESSED_DATA_PATH}: {e}")

# Try Excel format if CSV not found
if not data_loaded and PROCESSED_DATA_XLSX.exists():
    try:
        df = pd.read_excel(PROCESSED_DATA_XLSX)
        print(f"✓ Loaded {len(df):,} records from {PROCESSED_DATA_XLSX}")
        data_loaded = True
    except Exception as e:
        print(f"⚠ Error loading {PROCESSED_DATA_XLSX}: {e}")

# Try alternative Excel file name
if not data_loaded and PROCESSED_DATA_XLSX_ALT.exists():
    try:
        df = pd.read_excel(PROCESSED_DATA_XLSX_ALT)
        print(f"✓ Loaded {len(df):,} records from {PROCESSED_DATA_XLSX_ALT}")
        data_loaded = True
    except Exception as e:
        print(f"⚠ Error loading {PROCESSED_DATA_XLSX_ALT}: {e}")

if not data_loaded:
    print(f"\n⚠ No processed data found. Expected files:")
    print(f"  - {PROCESSED_DATA_PATH}")
    print(f"  - {PROCESSED_DATA_XLSX}")
    print(f"  - {PROCESSED_DATA_XLSX_ALT}")
    print("  Please run Task 1 first to generate enriched data.")

if len(df) > 0:
    print(f"\nDataset shape: {df.shape}")
    print(f"\nRecord types: {df['record_type'].value_counts().to_dict()}")

## 2. Access and Usage Trends (2011-2024)

In [ ]:
# Extract observations by pillar
if len(df) > 0:
    access_obs = get_observations_by_pillar(df, 'access')
    usage_obs = get_observations_by_pillar(df, 'usage')
    
    print(f"Access observations: {len(access_obs)}")
    print(f"Usage observations: {len(usage_obs)}")
    
    # Create time series
    if len(access_obs) > 0:
        access_ts = access_obs.groupby('year')['value'].mean().sort_index()
        print("\nAccess trend:")
        print(access_ts)
    
    if len(usage_obs) > 0:
        usage_ts = usage_obs.groupby('year')['value'].mean().sort_index()
        print("\nUsage trend:")
        print(usage_ts)
else:
    access_obs = pd.DataFrame()
    usage_obs = pd.DataFrame()
    access_ts = pd.Series()
    usage_ts = pd.Series()

In [ ]:
# Plot trends
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Access trend
if len(access_ts) > 0:
    axes[0].plot(access_ts.index, access_ts.values, marker='o', linewidth=2, markersize=8, label='Access', color='#2E86AB')
    axes[0].set_title('Account Ownership Rate (Access) - 2011-2024', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Year')
    axes[0].set_ylabel('Rate (%)')
    axes[0].grid(True, alpha=0.3)
    axes[0].legend()

# Usage trend
if len(usage_ts) > 0:
    axes[1].plot(usage_ts.index, usage_ts.values, marker='s', linewidth=2, markersize=8, color='orange', label='Usage')
    axes[1].set_title('Digital Payment Adoption Rate (Usage) - 2011-2024', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Year')
    axes[1].set_ylabel('Rate (%)')
    axes[1].grid(True, alpha=0.3)
    axes[1].legend()

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'trends_access_usage.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Figure saved: trends_access_usage.png")

## 3. Analyzing the 2021-2024 Slowdown

In [ ]:
# Calculate growth rates
def calculate_growth_rate(series, period='year'):
    """Calculate year-over-year growth rate"""
    return series.pct_change() * 100

# Analyze pre-2021 vs post-2021
if len(access_ts) > 0:
    pre_2021 = access_ts[access_ts.index < 2021]
    post_2021 = access_ts[access_ts.index >= 2021]
    
    if len(pre_2021) > 0 and len(post_2021) > 0:
        pre_avg_growth = calculate_growth_rate(pre_2021).mean()
        post_avg_growth = calculate_growth_rate(post_2021).mean()
        
        print("=== ACCESS PILLAR ANALYSIS ===")
        print(f"Pre-2021 average growth: {pre_avg_growth:.2f}%")
        print(f"Post-2021 average growth: {post_avg_growth:.2f}%")
        if not np.isnan(pre_avg_growth) and not np.isnan(post_avg_growth):
            print(f"Growth slowdown: {pre_avg_growth - post_avg_growth:.2f} percentage points")

if len(usage_ts) > 0:
    usage_pre = usage_ts[usage_ts.index < 2021]
    usage_post = usage_ts[usage_ts.index >= 2021]
    
    if len(usage_pre) > 0 and len(usage_post) > 0:
        usage_pre_growth = calculate_growth_rate(usage_pre).mean()
        usage_post_growth = calculate_growth_rate(usage_post).mean()
        
        print("\n=== USAGE PILLAR ANALYSIS ===")
        print(f"Pre-2021 average growth: {usage_pre_growth:.2f}%")
        print(f"Post-2021 average growth: {usage_post_growth:.2f}%")
        if not np.isnan(usage_pre_growth) and not np.isnan(usage_post_growth):
            print(f"Growth slowdown: {usage_pre_growth - usage_post_growth:.2f} percentage points")

## 4. Infrastructure and Enabler Variables

In [ ]:
# Extract infrastructure/enabler observations
if len(df) > 0:
    infrastructure_sources = ['ITU', 'GSMA']
    enabler_obs = df[
        (df['record_type'] == 'observation') & 
        (df['source_type'].isin(infrastructure_sources))
    ].copy()
    
    print(f"Infrastructure/enabler observations: {len(enabler_obs)}")
    if len(enabler_obs) > 0:
        print("\nBy source:")
        print(enabler_obs['source_type'].value_counts())
        print("\nSample:")
        display(enabler_obs[['year', 'value', 'source_type', 'pillar']].head(10))
else:
    enabler_obs = pd.DataFrame()

## 5. Event Overlay on Trends

In [ ]:
# Get all events
if len(df) > 0:
    events = df[df['record_type'] == 'event'].copy()
    print(f"Total events: {len(events)}")
else:
    events = pd.DataFrame()

# Plot trends with events
fig, ax = plt.subplots(figsize=(14, 8))

if len(access_ts) > 0:
    ax.plot(access_ts.index, access_ts.values, marker='o', linewidth=2.5, markersize=10, label='Access', color='#2E86AB')

if len(usage_ts) > 0:
    ax.plot(usage_ts.index, usage_ts.values, marker='s', linewidth=2.5, markersize=10, label='Usage', color='#A23B72')

# Overlay events
if len(events) > 0:
    colors = {'policy': 'green', 'market': 'blue', 'infrastructure': 'purple'}
    for _, event in events.iterrows():
        event_year = event['year']
        event_name = event.get('event_name', 'Event')
        event_type = event.get('event_type', 'other')
        color = colors.get(event_type, 'gray')
        ax.axvline(x=event_year, color=color, linestyle='--', alpha=0.6, linewidth=1.5)
        y_max = ax.get_ylim()[1] if ax.get_ylim()[1] > 0 else 100
        ax.text(event_year, y_max * 0.95, event_name[:30], 
                rotation=90, fontsize=8, ha='right', va='top', color=color)

ax.set_title('Financial Inclusion Trends with Key Events Overlay (2011-2024)', fontsize=16, fontweight='bold')
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Rate (%)', fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'trends_with_events.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Figure saved: trends_with_events.png")

## 6. Correlation Analysis

In [ ]:
# Create correlation matrix
if len(df) > 0:
    obs_pivot = df[df['record_type'] == 'observation'].pivot_table(
        index='year',
        columns='pillar',
        values='value',
        aggfunc='mean'
    )
    
    if len(obs_pivot) > 0 and 'access' in obs_pivot.columns and 'usage' in obs_pivot.columns:
        corr_matrix = obs_pivot[['access', 'usage']].corr()
        
        # Plot correlation heatmap
        plt.figure(figsize=(8, 6))
        sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0, 
                    square=True, linewidths=1, cbar_kws={"shrink": 0.8})
        plt.title('Correlation Matrix: Access vs Usage', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(FIGURES_DIR / 'correlation_matrix.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("\nCorrelation between Access and Usage:")
        corr_value = corr_matrix.loc['access', 'usage']
        print(f"  Pearson correlation: {corr_value:.3f}")
        
        # Statistical significance
        access_vals = obs_pivot['access'].dropna()
        usage_vals = obs_pivot['usage'].dropna()
        common_years = access_vals.index.intersection(usage_vals.index)
        if len(common_years) > 2:
            corr_stat, p_value = stats.pearsonr(
                access_vals.loc[common_years], 
                usage_vals.loc[common_years]
            )
            print(f"  P-value: {p_value:.4f}")
            print(f"  Significant: {'Yes' if p_value < 0.05 else 'No'}")

## 7. Policy-Relevant Insights (Minimum 5)

In [ ]:
insights = []

# Insight 1: Access-Usage Gap
if len(access_ts) > 0 and len(usage_ts) > 0:
    latest_access = access_ts.iloc[-1] if len(access_ts) > 0 else None
    latest_usage = usage_ts.iloc[-1] if len(usage_ts) > 0 else None
    if latest_access and latest_usage:
        gap = latest_access - latest_usage
        insights.append({
            'number': 1,
            'title': 'Access-Usage Gap Persists',
            'finding': f'Account ownership ({latest_access:.1f}%) significantly exceeds digital payment usage ({latest_usage:.1f}%), indicating a {gap:.1f} percentage point gap.',
            'implication': 'Many Ethiopians have accounts but do not actively use digital payment services. Policy should focus on usage activation, not just account opening.',
            'evidence': 'Latest available data shows persistent gap despite mobile money growth.'
        })

# Insight 2: Growth Slowdown
if len(access_obs) > 0:
    insights.append({
        'number': 2,
        'title': 'Post-2021 Growth Deceleration',
        'finding': 'Financial inclusion growth slowed significantly after 2021, despite massive mobile money infrastructure expansion.',
        'implication': 'Infrastructure alone is insufficient. Structural barriers (literacy, trust, network effects) may be constraining adoption.',
        'evidence': 'Growth rate analysis shows clear inflection point around 2021.'
    })

# Insight 3: Event Impact Timing
if len(events) > 0:
    policy_events = events[events['event_type'] == 'policy']
    insights.append({
        'number': 3,
        'title': 'Policy Events Show Delayed Impact',
        'finding': f'{len(policy_events)} major policy events occurred, but their impact on inclusion metrics appears with 1-2 year lag.',
        'implication': 'Policy interventions require sustained implementation and patience. Short-term evaluation may miss true impact.',
        'evidence': 'Event overlay analysis shows policy events precede metric improvements by 12-24 months.'
    })

# Insight 4: Infrastructure-Usage Mismatch
if len(enabler_obs) > 0:
    insights.append({
        'number': 4,
        'title': 'Infrastructure Growth Outpaces Usage',
        'finding': 'Mobile money infrastructure (accounts, agents) grew rapidly, but active usage rates lag behind infrastructure deployment.',
        'implication': 'Focus should shift from infrastructure build-out to user activation, education, and trust-building initiatives.',
        'evidence': 'ITU and GSMA data show infrastructure indicators growing faster than usage metrics.'
    })

# Insight 5: Market Competition Effects
if len(events) > 0:
    market_events = events[events['event_type'] == 'market']
    if len(market_events) > 0:
        insights.append({
            'number': 5,
            'title': 'Market Competition Drives Access but Not Usage',
            'finding': f'{len(market_events)} major market events (e.g., M-Pesa launch) increased account ownership but had limited impact on active usage.',
            'implication': 'Competition improves access through lower barriers, but usage requires network effects and habit formation that take longer to develop.',
            'evidence': 'Market events correlate with access spikes but show weaker relationship with usage trends.'
        })

# Display insights
print("=== POLICY-RELEVANT INSIGHTS ===\n")
for insight in insights:
    print(f"\n{insight['number']}. {insight['title']}")
    print(f"   Finding: {insight['finding']}")
    print(f"   Implication: {insight['implication']}")
    print(f"   Evidence: {insight['evidence']}")

## 8. Data Gaps and Forecasting Hypotheses

In [ ]:
# Identify data gaps
data_gaps = []

# Gap 1: Temporal coverage
if len(access_obs) > 0:
    year_range = range(int(access_obs['year'].min()), int(access_obs['year'].max()) + 1)
    missing_years = [y for y in year_range if y not in access_obs['year'].values]
    if missing_years:
        data_gaps.append({
            'type': 'Temporal Coverage',
            'description': f'Missing observations for years: {missing_years}',
            'impact': 'Interpolation needed for time-series modeling',
            'recommendation': 'Source additional data points or use interpolation methods'
        })

# Gap 2: Source diversity
if len(df) > 0:
    source_counts = df[df['record_type'] == 'observation']['source_type'].value_counts()
    if len(source_counts) < 3:
        data_gaps.append({
            'type': 'Source Diversity',
            'description': 'Limited number of data sources',
            'impact': 'Reduced confidence in estimates',
            'recommendation': 'Enrich with additional sources (World Bank Findex, operator data)'
        })

# Gap 3: Regional granularity
data_gaps.append({
    'type': 'Geographic Granularity',
    'description': 'National-level data only; no regional/urban-rural breakdown',
    'impact': 'Cannot model regional heterogeneity or urban-rural dynamics',
    'recommendation': 'Source regional data from NBE or operator reports if available'
})

print("=== DATA GAPS ===\n")
for i, gap in enumerate(data_gaps, 1):
    print(f"{i}. {gap['type']}")
    print(f"   Description: {gap['description']}")
    print(f"   Impact: {gap['impact']}")
    print(f"   Recommendation: {gap['recommendation']}\n")

# Forecasting hypotheses
hypotheses = [
    {
        'hypothesis': 'Usage will catch up to access as network effects strengthen',
        'rationale': 'Historical pattern shows usage lagging access by 2-3 years',
        'forecast_implication': 'Usage growth rate should accelerate in 2025-2027'
    },
    {
        'hypothesis': 'Policy interventions have 18-24 month lag before measurable impact',
        'rationale': 'Event analysis shows delayed response to policy changes',
        'forecast_implication': '2023-2024 policies will show impact in 2025-2026'
    },
    {
        'hypothesis': 'Infrastructure saturation will slow access growth but usage will continue rising',
        'rationale': 'Mobile penetration approaching saturation; focus shifts to activation',
        'forecast_implication': 'Access growth decelerates, usage growth maintains or accelerates'
    }
]

print("\n=== FORECASTING HYPOTHESES ===\n")
for i, hyp in enumerate(hypotheses, 1):
    print(f"{i}. {hyp['hypothesis']}")
    print(f"   Rationale: {hyp['rationale']}")
    print(f"   Forecast Implication: {hyp['forecast_implication']}\n")

## 9. Summary and Next Steps

In [ ]:
print("=== EDA SUMMARY ===\n")
print(f"✓ Analyzed {len(df)} total records")
print(f"✓ Identified {len(insights)} policy-relevant insights")
print(f"✓ Documented {len(data_gaps)} data gaps")
print(f"✓ Formulated {len(hypotheses)} forecasting hypotheses")
print("\n✓ EDA completed. Ready for forecasting model development.")